In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timezone, timedelta
import pytz
import re
import pandas as pd
import time
import numpy as np
import sys
import os
import matplotlib
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 70)

In [3]:
def num_null_rows_in_df(df: pd.DataFrame, exempt_cols):
    total_rows = df.shape[0]
    for col in df.columns:
        if df[col].isnull().any() and col not in exempt_cols:
            count_null_row = len([match for match in df[col].isnull() if match == True])
            print(f"{col}: {count_null_row} null rows, {count_null_row/total_rows*100:.1f}%")
    print(f"total_rows: {total_rows}")


def num_null_rows_in_col(df: pd.DataFrame, col: str):
    total_rows = df.shape[0]

    if df[col].isnull().any():
        count_null_row = len([match for match in df[col].isnull() if match == True])
        print(f"{col}: {count_null_row} null rows, {count_null_row/total_rows*100:.1f}%")
    else:
        print('col not null')

    print(f"total_rows: {total_rows}")


def drop_all_null_col(match_stats_summary):
    all_null_cols = []
    for col in match_stats_summary.columns:
        if match_stats_summary[col].isnull().all():
            all_null_cols.append(col)
    print(all_null_cols)
    return match_stats_summary.drop(columns=all_null_cols)


matches_with_issues = ['https://fbref.com/en/matches/a62fa3bc/Ternana-Brescia-October-3-2017-Serie-B',]


def drop_and_clean(df):
    df = df[~df['link'].isin(matches_with_issues)]
    return df.drop_duplicates()

In [4]:
match_directory = 'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\matches'
location_directory = 'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\venue_location_data'
# core_cols = mega_match_file.drop(columns=['home_player_stats', 'away_player_stats', 'match_events', 'officials', 'shots', 'overall_stats'])
core_cols = ['away_npxG', 'away_team', 'home_goals', 'home_team', 'home_xAG', 'home_team_id', 'league', 'utc_datetime', 'home_ast', 'local_date', 'away_team_id', 'link', 'away_formation', 'away_ast', 'home_manager',
             'stage', 'year', 'attendance', 'away_xAG', 'second_half_stoppage', 'away_goals', 'home_npxG', 'away_manager', 'first_half_stoppage', 'home_captain', 'away_captain', 'home_formation', 'home_xG', 'away_xG', 'venue']
mega_venue_file = pd.read_parquet(location_directory + '\\all_venue_data.parquet')
match_data_file_list = [data_file for data_file in os.listdir(match_directory)
                        if '.pkl' in data_file]

In [5]:
location_data_file_list = [data_file for data_file in os.listdir(location_directory)
                           if '.parquet' in data_file]
mega_venue_file = pd.read_parquet(location_directory + '\\all_venue_data.parquet')

# mega_venue_file = pd.DataFrame()
# for file in location_data_file_list:
#     mega_venue_file = pd.concat([mega_venue_file, pd.read_parquet(f"{location_directory}\\{file}")], ignore_index=True)


# display(mega_venue_file.head())
# mega_venue_file.shape
# mega_venue_file.to_parquet(location_directory + '\\all_venue_data.parquet')

In [12]:
match_info = pd.read_pickle('C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\all_match_data.pkl')
match_info.rename(columns={'home_passing_accuracy': 'home_pass_num', 'away_passing_accuracy': 'away_pass_num'}, inplace=True)
# match_info = pd.DataFrame()

# for file in match_data_file_list:
#     match_info = pd.concat([match_info, pd.read_pickle(f"{match_directory}\\{file}")], ignore_index=True)

In [ ]:
def split_off_core_rows(match_info):
    match_info_core = match_info[core_cols]
    match_info_core = match_info_core.infer_objects()
    overall_stats_mask = match_info['overall_stats'].apply(lambda x: x.shape[1] > 1)
    matches_with_overall_stats = match_info.loc[overall_stats_mask, 'overall_stats'].apply(lambda x: x.iloc[0])
    match_info_core = match_info_core.merge(matches_with_overall_stats, left_index=True, right_index=True, how='outer')
    match_info_core['year'] = pd.to_numeric(match_info_core['year'])
    match_info_core = match_info_core.drop_duplicates('link').reset_index(drop=True)
    match_info_core.loc[
        match_info_core['year'].isnull(), 'year'] = match_info_core.loc[
        match_info_core['year'].isnull(), 'local_date'].item().year
    match_info_core['local_date'] = match_info_core['local_date'].apply(pd.to_datetime)
    match_info_core['og_index'] = match_info_core.index
    return match_info_core
    # match_info_core.to_parquet('C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\core_match_data.parquet')

In [ ]:
# match_info_core = split_off_core_rows(match_info)

In [ ]:
# match_info_core.to_parquet('C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\core_match_data.parquet')
# match_info.to_pickle('C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\all_match_data.pkl')

In [6]:
def specific_value_correction(df: pd.DataFrame):

    df.loc[(df['link'] == 'https://fbref.com/en/matches/a5c789b7/Estrela-Moreirense-November-11-2023-Primeira-Liga'),
           'league'] = 'Primeira Liga'

    df.loc[(df['link'] == 'https://fbref.com/en/matches/e0a20cfe/Hellas-Verona-Roma-September-19-2020-Serie-A'),
           'home_goals'] = 3.0

    df.loc[(df['link'] == 'https://fbref.com/en/matches/abb94d2f/Alcorcon-Zaragoza-October-3-2020-Segunda-Division'),
           'away_goals'] = 0.0
    df.loc[(df['link'] == 'https://fbref.com/en/matches/cbab563a/Bordeaux-Rodez-Aveyron-June-2-2023-Ligue-2'),
           'away_goals'] = 1.0
    df.loc[(df['link'] == 'https://fbref.com/en/matches/a6ce6043/Brescia-Cosenza-June-1-2023-Serie-B'),
           'away_goals'] = 3.0

    df.loc[(df['link'] == 'https://fbref.com/en/matches/c6950828/Osnabruck-Magdeburg-November-25-2023-2-Bundesliga'),
           'home_manager'] = 'sacked'
    df.loc[(df['link'] == 'https://fbref.com/en/matches/2019ab7a/Zwolle-Heerenveen-February-26-2021-Eredivisie'),
           'home_manager'] = 'sacked'
    df.loc[(df['link'] == 'https://fbref.com/en/matches/8f6f1fca/Extremadura-Zaragoza-May-11-2019-Segunda-Division'),
           'home_manager'] = df.loc[20180, 'home_manager']
    df.loc[(df['link'] == 'https://fbref.com/en/matches/75ba4cd0/Elche-Deportivo-La-Coruna-June-4-2019-Segunda-Division'),
           'home_manager'] = df.loc[20209, 'home_manager']
    df.loc[(df['link'] == 'https://fbref.com/en/matches/4e350853/FC-Andorra-Eldense-November-12-2023-Segunda-Division'),
           'home_manager'] = df.loc[22240, 'home_manager']

    df.loc[(df['link'] == 'https://fbref.com/en/matches/38315c82/Malaga-Elche-June-8-2019-Segunda-Division'),
           'away_manager'] = df.loc[20177, 'away_manager']
    df.loc[(df['link'] == 'https://fbref.com/en/matches/b2607350/Santa-Clara-Feirense-May-11-2019-Primeira-Liga'),
           'away_manager'] = df.loc[34149, 'away_manager']

    return df


def update_null_venues(match_info_core, col='venue'):
    # not generic but could be
    null_mask = match_info_core['venue'].isnull()
    season_venue_info = match_info_core.loc[~null_mask][['home_team', 'year', 'venue', 'league', 'local_date']].drop_duplicates()

    season_venue_info = season_venue_info.loc[season_venue_info['venue'].notnull()]
    home_teams_missing = match_info_core.loc[null_mask][['home_team', 'year', 'local_date', 'venue', 'league']].drop_duplicates()
    intermediate_df = home_teams_missing.merge(season_venue_info, suffixes=('_null', '_info'), on=['home_team',])

    outdated_null_teams = intermediate_df['home_team'].drop_duplicates()
    intermediate_df['info_dist'] = abs(intermediate_df['local_date_info'] - intermediate_df['local_date_null']).astype('int64') * 1.1574e-14  # want info as days
    intermediate_df = intermediate_df.drop_duplicates(['home_team', 'local_date_null']).reset_index(drop=True)

    correct_rows = pd.DataFrame()
    for team in outdated_null_teams:
        tmp = intermediate_df[intermediate_df['home_team'] == team]
        for null_date in tmp['local_date_null'].drop_duplicates():

            date_mask = tmp['local_date_null'] == null_date
            min_mask = tmp.loc[date_mask, 'info_dist'].min() == tmp['info_dist']
            correct_rows = pd.concat([correct_rows, tmp.loc[min_mask]])

    if correct_rows.shape[0] > 0:
        correct_rows = correct_rows[['home_team', 'local_date_null', 'venue_info']].rename(columns={'local_date_null': 'local_date', 'venue_info': 'venue'})
        tmp = match_info_core.loc[null_mask][['home_team', 'local_date']]
        tmp['old_index'] = tmp.index
        rows_to_replace = tmp.merge(correct_rows, how='inner', on=['home_team', 'local_date'], validate='1:1',)
        match_info_core = match_info_core.merge(rows_to_replace[['old_index', 'venue']].rename(
            columns={'venue': 'null_venue_info'}), left_index=True, right_on=['old_index'], how='left', validate='1:1')
        match_info_core.loc[match_info_core['venue'].isnull(), 'venue'] = match_info_core.loc[match_info_core['venue'].isnull(), 'null_venue_info']
        match_info_core = match_info_core.drop(columns=['old_index', 'null_venue_info']).reset_index(drop=True)
    return match_info_core


def split_home_away(df: pd.DataFrame, rename: bool = True) -> tuple[pd.DataFrame, pd.DataFrame]:
    essential_cols = ['utc_datetime', 'local_date', 'link', 'venue', 'year', 'league',]
    home_cols = ['attendance']
    away_cols = []
    home_df = df[[col for col in df.columns if ('home' in col or 'stoppage' in col or col in essential_cols or col in home_cols)]]
    away_df = df[[col for col in df.columns if ('away' in col or 'stoppage' in col or col in essential_cols or col in away_cols)]]
    if rename:
        home_df.columns = home_df.columns.str.replace('home_', '')
        away_df.columns = away_df.columns.str.replace('away_', '')

    return home_df, away_df


def homogenize_home_away(home_df: pd.DataFrame, away_df: pd.DataFrame) -> pd.DataFrame:

    home_df.columns = home_df.columns.str.replace('home_', '')
    away_df.columns = away_df.columns.str.replace('away_', '')
    return pd.concat([home_df, away_df], ignore_index=True).sort_values('local_date', ascending=False).reset_index(drop=True)

In [ ]:
match_info.head(1)

In [13]:
link_index_map = match_info['link'].to_dict()

In [14]:
officials_mask = match_info['officials'].apply(len) > 0
g = match_info.loc[officials_mask, 'officials']
officials = g.apply(pd.Series)
link_index_map = match_info['link'].to_dict()
officials['link'] = officials.index.map(link_index_map)
# officials_save = officials.copy()
# officials = officials_save.copy()

In [15]:
def fix_missing_ref_type(officials):
    col = 1
    paren_mask = officials[col].str.contains('\(|\)')
    not_null_mask = officials[col].notnull()
    mask_1 = (~((paren_mask) & (not_null_mask)) & not_null_mask)
    copy_mask = mask_1
    officials.loc[copy_mask, 1] = officials.loc[copy_mask, 1] + ' ' + officials.loc[copy_mask, 2]
    officials.loc[copy_mask, 2] = officials.loc[copy_mask, 3]
    officials.loc[copy_mask, 3] = officials.loc[copy_mask, 4]
    officials.loc[copy_mask, 4] = officials.loc[copy_mask, 5]
    officials.loc[copy_mask, 5] = np.NaN

    col = 2
    paren_mask = officials[col].str.contains('\(|\)')
    not_null_mask = officials[col].notnull()
    mask_2 = (~((paren_mask) & (not_null_mask)) & not_null_mask)
    copy_mask = mask_2
    officials.loc[copy_mask, 2] = officials.loc[copy_mask, 2] + ' ' + officials.loc[copy_mask, 3]
    officials.loc[copy_mask, 3] = officials.loc[copy_mask, 4]
    officials.loc[copy_mask, 4] = officials.loc[copy_mask, 5]
    officials.loc[copy_mask, 5] = np.NaN
    return officials


def seperate_ref_types(officials):

    col = 1
    str_mask = officials[col].str.contains('AR1')
    not_AR1_mask = ~((str_mask).fillna(False))
    officials.loc[~not_AR1_mask, 'AR1'] = officials.loc[~not_AR1_mask, col]
    officials.loc[not_AR1_mask, '4th'] = officials.loc[not_AR1_mask, col]

    col = 2
    str_mask = officials[col].str.contains('AR2')
    not_AR2_mask = ~((str_mask).fillna(False))
    officials.loc[~not_AR2_mask, 'AR2'] = officials.loc[~not_AR2_mask, col]
    officials.loc[not_AR2_mask, 'VAR'] = officials.loc[not_AR2_mask, col]

    col = 3
    str_mask = officials[col].str.contains('4th')
    not_4th_mask = ~((str_mask).fillna(False))
    officials.loc[~not_4th_mask, '4th'] = officials.loc[~not_4th_mask, col]

    col = 4
    str_mask = officials[col].str.contains('VAR')
    not_VAR_mask = ~((str_mask).fillna(False))
    officials.loc[~not_VAR_mask, 'VAR'] = officials.loc[~not_VAR_mask, col]
    officials.rename(columns={0: 'referee', }, inplace=True)

    return officials


def validate_ref_info(officials):
    for ref in ['AR1', 'AR2', '4th', 'VAR']:
        for col in [1, 2, 3, 4, 5]:
            mask_1 = (officials[col].str.contains(ref)).fillna(False)
            mask_2 = (officials.loc[mask_1, col] != officials.loc[mask_1, ref])
            filt = officials.loc[mask_1]
            out = filt.loc[(mask_2)]
            if out.shape[0] > 0:
                display(f'ref: {ref} - col: {col}')
                display(out)
    print('finished validating, nothing displayed means no issues')

In [16]:
officials = fix_missing_ref_type(officials)
officials = seperate_ref_types(officials)
validate_ref_info(officials)

finished validating, nothing displayed means no issues


In [17]:
officials = officials[['link', 'referee', 'AR1', 'AR2', '4th', 'VAR']]
for col in ['referee', 'AR1', 'AR2', '4th', 'VAR']:
    officials[col] = officials[col].str.split('(', regex=False).str[0].str.strip()

In [18]:
officials.head()

,link,referee,AR1,AR2,4th,VAR
0,https://fbref.com/en/matches/214b4c53/Dusseldo...,Deniz Aytekin,Benjamin Brand,Marco Achmüller,Mike Pickel,NaN
1,https://fbref.com/en/matches/27a83d3d/RB-Leipz...,Christian Dingert,Aarne Aarnink,Torsten Bauer,Michael Emmer,NaN
2,https://fbref.com/en/matches/6228d604/Bochum-G...,Tobias Stieler,Florian Heft,Jonas Weickenmeier,Guido Kleve,NaN
3,https://fbref.com/en/matches/ad976a9d/St-Pauli...,Marco Fritz,Tobias Reichel,Dominik Schaal,Robert Schröder,NaN
4,https://fbref.com/en/matches/1f138e5f/Karlsruh...,Tobias Welz,Martin Thomsen,Rafael Foltyn,Martin Petersen,NaN


In [21]:
mega_venue_file.head()

,venue,latitude,longitude,name,links
0,"Allianz Arena, München",48.218808,11.624664,"Allianz Arena, Werner-Heisenberg-Allee 25, 809...",https://www.openstreetmap.org/?mlat=48.21881&m...
1,"PreZero Arena, Sinsheim",49.238060,8.887641,"PreZero Arena, Dietmar-Hopp-Straße 1, 74889 Si...",https://www.openstreetmap.org/?mlat=49.23806&m...
2,"HDI-Arena, Hannover",52.360026,9.731016,"Heinz-von-Heiden-Arena, Robert-Enke-Straße 3, ...",https://www.openstreetmap.org/?mlat=52.36003&m...
3,"RheinEnergieSTADION, Köln",50.933506,6.875117,"RheinEnergieStadion, Aachener Straße 999, 5093...",https://www.openstreetmap.org/?mlat=50.93351&m...
4,"Commerzbank-Arena, Frankfurt am Main",50.068599,8.645463,"Deutsche Bank Park, Mörfelder Landstraße 362, ...",https://www.openstreetmap.org/?mlat=50.06860&m...


In [38]:
match_info_core = pd.read_parquet('C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\core_match_data.parquet')
match_info_core.rename(columns={'home_passing_accuracy': 'home_pass_num', 'away_passing_accuracy': 'away_pass_num'}, inplace=True)


core_cols_w_stats = match_info_core.columns


match_info_core = update_null_venues(match_info_core)


match_info_core = specific_value_correction(match_info_core)

try:
    match_info_core = match_info_core.drop(columns=['referee', 'AR1', 'AR2', '4th', 'VAR']).merge(officials, on='link', validate='1:1')
except KeyError:
    match_info_core = match_info_core.merge(officials, on='link', validate='1:1')

match_info_core = match_info_core.merge(mega_venue_file.drop(columns=['name','links']), on = 'venue', validate='m:1')
# null_rows_in_col(match_info_core)

In [39]:
# the _ast are assists and indicate there are no player specific stats at all, mostly 2012 and 2013, pretty old data and can be removed
fully_drop_cols = ['home_saves_frac', 'away_saves_frac']
drop_null_cols = ['venue', 'home_goals', 'away_goals', 'league', 'home_ast', 'away_ast', 'home_formation', 'away_formation']
okay_null_cols = ['home_captain', 'away_captain', 'home_npxG', 'away_npxG', 'attendance', 'home_xAG', 'away_xAG', 'home_xG', 'away_xG', 'home_pass_num', 'away_pass_num',
                  'home_passing_accuracy_frac', 'away_passing_accuracy_frac', 'home_possession_frac', 'away_possession_frac', 'home_shots_on_target', 'away_shots_on_target', 'home_saves', 'away_saves']

avg_null_cols = ['utc_datetime', ]
infer_null_cols = []
# if there are no shots at all shots_on_target_frac will be NaN, but matches without this data will also be NaN
fill_null_cols = ['VAR', ]
fill_null_dict = {'VAR': 'No VAR',}
# fill_null_cols = ['VAR', 'home_shots_on_target_frac', 'away_shots_on_target_frac']
# fill_null_dict = {'VAR': 'No VAR','home_shots_on_target_frac': -0.01, 'away_shots_on_target_frac': -0.01, }

for col in drop_null_cols:
    match_info_core = match_info_core[~match_info_core[col].isnull()]
try:
    match_info_core = match_info_core.drop(columns=fully_drop_cols)
except KeyError:
    pass

In [40]:
g = match_info_core[match_info_core['home_shots_on_target_frac'].isnull()].copy()

In [41]:
g['year'] = g['utc_datetime'].astype(str).str[:4]
# g[['year','league']].value_counts()
gg = g[(g['home_shots_on_target'].isnull()) & (g['away_shots_on_target'].isnull())]
display(g['year'].value_counts())
gg[['year',]].value_counts()

year
2015    2228
2016    1837
2017    1481
2014    1295
NaT      760
2018     433
2019      22
2020       2
2021       1
Name: count, dtype: int64

year
2015    2228
2016    1837
2017    1481
2014    1295
NaT      760
2018     433
2019      21
Name: count, dtype: int64

In [46]:
# match_info_core[(match_info_core['league']== 'Ligue 2') & (match_info_core['utc_datetime'].astype(str).str[:4] == '2019')]

In [47]:
# gg[gg['year'] == '2019']

In [25]:
match_info_core.head()

,away_npxG,away_team,home_goals,home_team,home_xAG,home_team_id,league,utc_datetime,home_ast,local_date,away_team_id,link,away_formation,away_ast,home_manager,stage,year,attendance,away_xAG,second_half_stoppage,away_goals,home_npxG,away_manager,first_half_stoppage,home_captain,away_captain,home_formation,home_xG,away_xG,venue,home_possession_frac,away_possession_frac,home_pass_num,away_pass_num,home_passing_accuracy_frac,away_passing_accuracy_frac,home_shots_on_target,away_shots_on_target,home_shots_on_target_frac,away_shots_on_target_frac,home_saves,away_saves,og_index,referee,AR1,AR2,4th,VAR,latitude,longitude
0,NaN,Eintracht Braunschweig,2.0,Düsseldorf,NaN,b1278397,Bundesliga 2,2014-08-01 18:30:00+00:00,2.0,2014-08-01,8107958d,https://fbref.com/en/matches/214b4c53/Dusseldo...,4-1-4-1,1.0,Oliver Reck,Matchweek 1,2014.0,"41,667",NaN,0,2.0,NaN,Torsten Lieberknecht,1,None,None,4-2-3-1,NaN,NaN,"Merkur Spielarena, Düsseldorf",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Deniz Aytekin,Benjamin Brand,Marco Achmüller,Mike Pickel,NaN,51.261788,6.733213
1,NaN,Aalen,0.0,RB Leipzig,NaN,acbb6a5b,Bundesliga 2,2014-08-02 11:00:00+00:00,0.0,2014-08-02,eb207015,https://fbref.com/en/matches/27a83d3d/RB-Leipz...,4-3-3,0.0,Alexander Zorniger,Matchweek 1,2014.0,"21,354",NaN,0,0.0,NaN,Stephan Ruthenbeck,0,None,None,4-1-2-1-2,NaN,NaN,"Red Bull Arena, Leipzig",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Christian Dingert,Aarne Aarnink,Torsten Bauer,Michael Emmer,NaN,51.345689,12.348376
2,NaN,Greuther Fürth,1.0,Bochum,NaN,b42c6323,Bundesliga 2,2014-08-02 13:30:00+00:00,0.0,2014-08-02,12192a4c,https://fbref.com/en/matches/6228d604/Bochum-G...,4-2-3-1,0.0,Peter Neururer,Matchweek 1,2014.0,"18,146",NaN,1,1.0,NaN,Frank Kramer,0,None,None,4-4-2,NaN,NaN,"Vonovia Ruhrstadion, Bochum",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Tobias Stieler,Florian Heft,Jonas Weickenmeier,Guido Kleve,NaN,51.490083,7.236509
3,NaN,Ingolstadt 04,1.0,St. Pauli,NaN,54864664,Bundesliga 2,2014-08-02 13:30:00+00:00,0.0,2014-08-02,12eb2039,https://fbref.com/en/matches/ad976a9d/St-Pauli...,4-3-3,0.0,Roland Vrabec,Matchweek 1,2014.0,"26,664",NaN,0,1.0,NaN,Ralph Hasenhüttl,0,None,None,4-4-2,NaN,NaN,"Millerntor-Stadion, Hamburg",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Marco Fritz,Tobias Reichel,Dominik Schaal,Robert Schröder,NaN,53.554557,9.967787
4,NaN,Union Berlin,0.0,Karlsruher,NaN,33ba9d7b,Bundesliga 2,2014-08-03 13:30:00+00:00,0.0,2014-08-03,7a41008f,https://fbref.com/en/matches/1f138e5f/Karlsruh...,3-5-2,0.0,Markus Kauczinski,Matchweek 1,2014.0,"18,489",NaN,0,0.0,NaN,Norbert Düwel,0,None,None,4-1-4-1,NaN,NaN,"Wildparkstadion, Karlsruhe",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Tobias Welz,Martin Thomsen,Rafael Foltyn,Martin Petersen,NaN,49.020061,8.413003


In [ ]:
def pull_home_away_stats(match_info,):
    cols = ['home_player_stats', 'away_player_stats']

    out_stats = pd.DataFrame()
    for stats_col in cols:
        _stat_mask = match_info[stats_col].apply(lambda x: (x.shape[1] > 1) if x is not None else False)
        _stat_level = match_info.loc[_stat_mask, stats_col].apply(lambda x: x.iloc[0])
        _stat_level['link'] = match_info.loc[_stat_mask, 'link']
        _stat_level['local_date'] = match_info.loc[_stat_mask, 'local_date']
        _stat_level['side'] = stats_col[:4]
        _stat_level['team'] = match_info.loc[_stat_mask, stats_col[:5] + 'team']
        out_stats = pd.concat([out_stats, _stat_level], ignore_index=True)

    out_stats = out_stats.sort_values('local_date', ascending=False).reset_index(drop=True)
    return out_stats

In [ ]:
stats = pull_home_away_stats(match_info)

In [ ]:
stats['team'].value_counts(dropna=False)

In [ ]:
def split_off_embedded_dfs(match_info, df_col, ):
    event_info = match_info[df_col]
    event_info = event_info[(event_info.notnull())]
    cur_col_mask = event_info.apply(len) > 0
    g = pd.concat([
        pd.concat([
            pd.Series(tup[0], name='orig_index', index=range(tup[1].shape[0])), tup[1],], axis=1)
        for tup in event_info.loc[cur_col_mask].items()
    ], ignore_index=True)
    return g
    # g.to_parquet('C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\events_all_matches.parquet')

In [ ]:
stats.columns

In [ ]:
stats_link_index_map = stats['link'].to_dict()
stats_side_index_map = stats['side'].to_dict()
stats_team_index_map = stats['team'].to_dict()

In [ ]:
temp = stats['summary'].apply(pd.DataFrame)

In [ ]:
temp['link'] = temp.index.map(stats_link_index_map)

In [ ]:
display(temp.shape)
temp.head()

In [ ]:
bench = pd.DataFrame()
bench['bench'] = stats['bench'].apply(pd.DataFrame)
key = 'bench'
# bench.to_parquet(f'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\{key}_all_matches.parquet')

In [ ]:
bench = split_off_embedded_dfs(bench, 'bench')

In [ ]:
out_dict = {}
keys = ['starting_lineup', 'summary', 'gk', 'passing', 'pass_types', 'defense', 'possession', 'misc',]
for key in keys:
    print(key)
    out_dict[key] = split_off_embedded_dfs(stats, key, stats_link_index_map, stats_side_index_map, stats_side_index_map)
    out_dict[key]['link'] = out_dict[key]['orig_index'].map(stats_link_index_map)
    out_dict[key]['team'] = out_dict[key]['orig_index'].map(stats_side_index_map)
    out_dict[key]['side'] = out_dict[key]['orig_index'].map(stats_team_index_map)
    out_dict[key].to_parquet(f'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\{key}_all_matches.parquet')

In [ ]:
match_link_index_map = match_info['link'].to_dict()

In [ ]:
other_keys = ['shots', 'match_events']
for key in other_keys:
    print(key)
    out_dict[key] = split_off_embedded_dfs(match_info, key, match_link_index_map, None, None)
    out_dict[key].to_parquet(f'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\{key}_all_matches.parquet')

In [5]:
def validate_merged_data(df, test_cols, debug=False):
    for col in test_cols:
        if debug:
            print(col)
        existed_col = col.replace('_drop', '')

        if (df[col] != df[existed_col]).any():
            mismatched_df = df[df[col] != df[existed_col]]

            if not mismatched_df[col].isnull().all():
                return mismatched_df, col

    return None, None

In [51]:
project_folder = 'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\'
data_folder = project_folder + 'data_files\\'
merged_folder = data_folder + 'merged_files\\'


In [ ]:
# raw_shots = drop_and_clean(pd.read_parquet(merged_folder + 'shots_all_matches.parquet'))
core_match_data = drop_and_clean(pd.read_parquet(merged_folder + 'core_match_data.parquet'))
player_stats = drop_and_clean(pd.read_parquet(merged_folder + 'summary_all_matches.parquet').rename(columns={"('Unnamed: 0_level_0', 'Player')": 'stats_player'}))

all_stats = player_stats
match_summary_mask = all_stats['stats_player'].str.contains('[0-9]')
match_stats_summary = all_stats[match_summary_mask]
all_stats = all_stats[~match_summary_mask]
unread_stats_files = ['gk', 'passing', 'pass_types', 'defense', 'possession', 'misc',]
the_files = {key: f'{key}_all_matches.parquet' for key in unread_stats_files}
# raw_shots.head()
merge_key = ['orig_index', 'stats_player', "('player_id', '')"]

for key in the_files.keys():
    print(key)
    current_file = drop_and_clean(pd.read_parquet(merged_folder + the_files[key]).rename(columns={"('Unnamed: 0_level_0', 'Player')": 'stats_player'}))
    match_summary_mask = current_file['stats_player'].str.contains('[0-9]')
    tmp_summary = current_file[match_summary_mask]
    current_file = current_file[~match_summary_mask]
    all_stats = all_stats.merge(current_file, on=merge_key, how='outer', indicator=key, suffixes=('', '_drop'), validate='1:1')
    match_stats_summary = match_stats_summary.merge(tmp_summary, on=merge_key, how='outer', indicator=key, suffixes=('', '_drop'), validate='1:1')

    # Trial and error shows the misc stats file has data the summary file does not
    # keep the column from the original data set for simplicity
    if key == 'misc':
        update_cols = ["('Performance', 'Fls')", "('Performance', 'Fld')", "('Performance', 'Off')", 
                       "('Performance', 'Crs')", "('Performance', 'TklW')", "('Performance', 'PKwon')","('Performance', 'PKcon')",
                       "('Performance', 'OG')"]
        for col in update_cols:
            other_col = col + '_drop'
            update_mask = all_stats[col].isnull() & all_stats[other_col].notnull()
            update_mask_summary = match_stats_summary[col].isnull() & match_stats_summary[other_col].notnull()
            all_stats.loc[update_mask, col] = all_stats.loc[update_mask, other_col]
            match_stats_summary.loc[update_mask_summary, col] = match_stats_summary.loc[update_mask_summary, other_col]
    drop_cols = [col for col in all_stats.columns if '_drop' in col]
    mismatched_df = None
    mismatched_df, col = validate_merged_data(all_stats, drop_cols)
    if mismatched_df is not None:
        raise Exception(f"value_mismatch in {col}, table {key}")
    mismatched_df, col = validate_merged_data(match_stats_summary, drop_cols)
    if mismatched_df is not None:
        raise Exception(f"value_mismatch in {col}, table {key}")
    all_stats.drop(columns=drop_cols + [key], inplace=True)
    match_stats_summary.drop(columns=drop_cols + [key], inplace=True)
    print(f"dropped cols are {[col.replace('_drop','') for col in drop_cols]}")

match_stats_summary = drop_all_null_col(match_stats_summary)
all_stats = drop_all_null_col(all_stats)

In [52]:
final_files = data_folder + "final_files\\"

In [ ]:
match_stats_summary.to_parquet(final_files + 'player_stats_summary.parquet')
all_stats.to_parquet(final_files + 'individual_player_stats.parquet')

In [53]:
match_info_core.to_parquet(final_files + 'tmp_match_info_core.parquet')

In [48]:
match_info_core.head()

,away_npxG,away_team,home_goals,home_team,home_xAG,home_team_id,league,utc_datetime,home_ast,local_date,away_team_id,link,away_formation,away_ast,home_manager,stage,year,attendance,away_xAG,second_half_stoppage,away_goals,home_npxG,away_manager,first_half_stoppage,home_captain,away_captain,home_formation,home_xG,away_xG,venue,home_possession_frac,away_possession_frac,home_pass_num,away_pass_num,home_passing_accuracy_frac,away_passing_accuracy_frac,home_shots_on_target,away_shots_on_target,home_shots_on_target_frac,away_shots_on_target_frac,home_saves,away_saves,og_index,referee,AR1,AR2,4th,VAR,latitude,longitude
0,NaN,Eintracht Braunschweig,2.0,Düsseldorf,NaN,b1278397,Bundesliga 2,2014-08-01 18:30:00+00:00,2.0,2014-08-01,8107958d,https://fbref.com/en/matches/214b4c53/Dusseldo...,4-1-4-1,1.0,Oliver Reck,Matchweek 1,2014.0,"41,667",NaN,0,2.0,NaN,Torsten Lieberknecht,1,None,None,4-2-3-1,NaN,NaN,"Merkur Spielarena, Düsseldorf",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Deniz Aytekin,Benjamin Brand,Marco Achmüller,Mike Pickel,NaN,51.261788,6.733213
1,NaN,Aalen,0.0,RB Leipzig,NaN,acbb6a5b,Bundesliga 2,2014-08-02 11:00:00+00:00,0.0,2014-08-02,eb207015,https://fbref.com/en/matches/27a83d3d/RB-Leipz...,4-3-3,0.0,Alexander Zorniger,Matchweek 1,2014.0,"21,354",NaN,0,0.0,NaN,Stephan Ruthenbeck,0,None,None,4-1-2-1-2,NaN,NaN,"Red Bull Arena, Leipzig",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Christian Dingert,Aarne Aarnink,Torsten Bauer,Michael Emmer,NaN,51.345689,12.348376
2,NaN,Greuther Fürth,1.0,Bochum,NaN,b42c6323,Bundesliga 2,2014-08-02 13:30:00+00:00,0.0,2014-08-02,12192a4c,https://fbref.com/en/matches/6228d604/Bochum-G...,4-2-3-1,0.0,Peter Neururer,Matchweek 1,2014.0,"18,146",NaN,1,1.0,NaN,Frank Kramer,0,None,None,4-4-2,NaN,NaN,"Vonovia Ruhrstadion, Bochum",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Tobias Stieler,Florian Heft,Jonas Weickenmeier,Guido Kleve,NaN,51.490083,7.236509
3,NaN,Ingolstadt 04,1.0,St. Pauli,NaN,54864664,Bundesliga 2,2014-08-02 13:30:00+00:00,0.0,2014-08-02,12eb2039,https://fbref.com/en/matches/ad976a9d/St-Pauli...,4-3-3,0.0,Roland Vrabec,Matchweek 1,2014.0,"26,664",NaN,0,1.0,NaN,Ralph Hasenhüttl,0,None,None,4-4-2,NaN,NaN,"Millerntor-Stadion, Hamburg",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Marco Fritz,Tobias Reichel,Dominik Schaal,Robert Schröder,NaN,53.554557,9.967787
4,NaN,Union Berlin,0.0,Karlsruher,NaN,33ba9d7b,Bundesliga 2,2014-08-03 13:30:00+00:00,0.0,2014-08-03,7a41008f,https://fbref.com/en/matches/1f138e5f/Karlsruh...,3-5-2,0.0,Markus Kauczinski,Matchweek 1,2014.0,"18,489",NaN,0,0.0,NaN,Norbert Düwel,0,None,None,4-1-4-1,NaN,NaN,"Wildparkstadion, Karlsruhe",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Tobias Welz,Martin Thomsen,Rafael Foltyn,Martin Petersen,NaN,49.020061,8.413003


In [ ]:
out_dict = {}
keys = ['starting_lineup', 'bench', 'summary', 'gk', 'passing', 'pass_types', 'defense', 'possession', 'misc', 'shots', 'match_events',]
for key in keys:
    print(key)
    out_dict[key] = pd.read_parquet(f'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\merged_files\\{key}_all_matches.parquet')

In [ ]:
out_dict['starting_lineup']['side'].value_counts(dropna=False)

In [ ]:
# stats_link_index_map[out_dict['starting_lineup']['orig_index'].max()]
stats_link_index_map[58591]

In [ ]:
out_dict['starting_lineup'][(out_dict['starting_lineup']['orig_index'] == 58591) | (out_dict['starting_lineup']['orig_index'] == 58590)]

In [ ]:
out_dict['starting_lineup']['orig_index'].value_counts(dropna=False)

In [ ]:
out_dict['starting_lineup'][['link', 'side']].value_counts(dropna=False)

In [ ]:
match_info[match_info['link'] == 'https://fbref.com/en/matches/59e4c4ee/ADO-Den-Haag-PSV-Eindhoven-May-17-2015-Eredivisie']['home_player_stats'].iloc[0]['starting_lineup'][0]

In [ ]:
help(pd.Series.value_counts)

In [ ]:
for key in out_dict.keys():
    print(key)
    display(out_dict[key].head())

In [ ]:
link_key = {
    'starting_lineup': 'link',
    'bench': 'link',
    'summary': ('link', ''),
    'gk': ('link', ''),
    'passing': ('link', ''),
    'pass_types': ('link', ''),
    'defense': ('link', ''),
    'possession': ('link', ''),
    'misc': ('link', ''),
    'shots': ('link', ''),
    'match_events': 'link',
}

In [ ]:
object_cols = ['overall_stats', ]
overall_stats_mask = match_info['overall_stats'].apply(lambda x: x.shape[1] > 1)


matches_with_overall_stats = match_info.loc[overall_stats_mask, 'overall_stats'].apply(lambda x: x.iloc[0])
match_info_core = match_info_core.merge(matches_with_overall_stats, left_index=True, right_index=True, how='outer')

In [ ]:
object_cols = ['overall_stats', ]
overall_stats_mask = match_info['overall_stats'].apply(lambda x: x.shape[1] > 1)

matches_with_overall_stats = match_info.loc[overall_stats_mask, 'overall_stats'].apply(lambda x: x.iloc[0])
match_info_core = match_info_core.merge(matches_with_overall_stats, left_index=True, right_index=True, how='outer')

In [ ]:
form_null.overall_stats

In [ ]:
object_cols = ['']
form_null.head()['home_player_stats'].apply(lambda x: x.iloc[0]['starting_lineup'])

In [ ]:

null_cols = ['away_formation']
cols_in_progress = [col for col in null_cols if col not in (drop_null_cols + okay_null_cols)]
max_iter = 25


not_null_mask = ((match_info_core['home_formation'].isnull()) | (match_info_core['away_formation'].isnull()))
tmp = match_info_core[not_null_mask].head(max_iter)

display(tmp[['home_team', 'away_team', 'local_date', 'year'] + null_cols])
display([row['link'] for _, row in tmp.iterrows()])

In [34]:
home_team, away_team = split_home_away(match_info_core)
team_tendencies = homogenize_home_away(home_team, away_team)


In [35]:
team_tendencies.head()

,goals,team,xAG,team_id,league,utc_datetime,ast,local_date,link,manager,year,attendance,second_half_stoppage,npxG,first_half_stoppage,captain,formation,xG,venue,possession_frac,pass_num,passing_accuracy_frac,shots_on_target,shots_on_target_frac,saves
0,2.0,PSV Eindhoven,3.1,e334d850,Eredivisie,2023-12-07 17:45:00+00:00,1.0,2023-12-07,https://fbref.com/en/matches/f8eec54a/PSV-Eind...,Peter Bosz,2023.0,"34,000",1,5.1,0,Luuk de Jong,4-3-3,5.1,"Philips Stadion, Eindhoven",0.61,629.0,0.84,30.0,0.27,1.0
1,0.0,Heerenveen,0.3,193ff7aa,Eredivisie,2023-12-07 17:45:00+00:00,0.0,2023-12-07,https://fbref.com/en/matches/f8eec54a/PSV-Eind...,Kees van Wonderen,2023.0,NaN,1,0.6,0,Sven van Beek,4-3-3,0.6,"Philips Stadion, Eindhoven",0.39,404.0,0.77,12.0,0.08,8.0
2,1.0,Volendam,0.0,8783f4ee,Eredivisie,2023-12-07 20:00:00+00:00,0.0,2023-12-07,https://fbref.com/en/matches/1cd4580a/Feyenoor...,Michael Dingsdag,2023.0,NaN,6,0.5,3,Xavier Mbuyamba,4-3-3,0.5,"Stadion Feijenoord, Rotterdam",0.17,165.0,0.39,2.0,1.00,10.0
3,3.0,Feyenoord,3.4,fb4ca611,Eredivisie,2023-12-07 20:00:00+00:00,3.0,2023-12-07,https://fbref.com/en/matches/1cd4580a/Feyenoor...,Arne Slot,2023.0,None,6,3.4,3,Gernot Trauner,4-2-3-1,3.4,"Stadion Feijenoord, Rotterdam",0.83,811.0,0.85,30.0,0.33,2.0
4,3.0,Ajax,0.5,19c3f8c4,Eredivisie,2023-12-06 19:00:00+00:00,2.0,2023-12-06,https://fbref.com/en/matches/640d61f8/RKC-Waal...,John van 't Schip,2023.0,NaN,4,0.6,1,Steven Bergwijn,4-3-3,1.4,"Mandemakers Stadion, Waalwijk",0.60,546.0,0.82,15.0,0.33,2.0


In [ ]:
team = 'Tours'
z = team_tendencies[(team_tendencies['team'] == team) & (team_tendencies['year'] == 2014)].sort_values('local_date', ascending=False)

z['formation'].value_counts()

In [ ]:
categories = ['team', 'year']
g = team_tendencies.groupby(categories)
generic_formation = g['formation'].agg(pd.Series.mode)
generic_formation = generic_formation.to_frame()

g = home_team.groupby(categories)
home_formation = g['formation'].agg(pd.Series.mode)
home_formation = home_formation.to_frame()

g = away_team.groupby(categories)
away_formation = g['formation'].agg(pd.Series.mode)
away_formation = away_formation.to_frame()

In [ ]:

compare_formation = pd.merge(left=home_formation, right=away_formation, left_index=True, right_index=True, suffixes=['_home', '_away'], validate='1:1')
infer_null_cols = ['formation_home', 'formation_away']
disp_df = pd.DataFrame()
for col in infer_null_cols:
    compare_formation[col]
    if disp_df.shape[0] == 0:
        disp_df = compare_formation[(compare_formation[col].isnull()) | (len(compare_formation[col].item()) == 0)]
    else:
        disp_df = pd.concat([disp_df, compare_formation[compare_formation[col].isnull()]], axis=1)

disp_df

In [ ]:
team_tendencies[team_tendencies['team'] == 'Virtus Lanciano']

In [ ]:

lookup = df.loc[[14256], 'link'].item()
df[df['link'] == lookup]

In [ ]:
understand and factor in formations

In [ ]:
validate substitutions

In [ ]:
check betting data suitability/validity

In [ ]:
pull extra betting criteriea

In [ ]:
create and discretize data sources for model
uniqueness of url shortened as key
- ideas for extra stats

In [ ]:
weather data? calculate distance between matches (modulate with time between matches or just let model do it)